In [1]:
%matplotlib inline
import os, struct
from array import array as pyarray
import numpy as np
from scipy.cluster.vq import *
from numpy import append, array, int8, uint8, zeros
from pylab import *
from numpy import *

In [2]:
from scipy.spatial import distance

class Data(object):
    # N points, k cluster, p dimensions
    def __init__(self, N, k, p):
        self.N = N
        self.k = k
        self.p = p
        self.true_center = []
        self.true_cluster_list = []
        self.data_set = []
        self.PIs = []
        self.means = []
        self.sigma = 0.0

    def init_board_gauss(self):
        n = float(self.N)/self.k
        X = []
        for i in range(self.k):
            c = ()
            # construct a random center point
            for j in range(self.p):
                c = c + (random.uniform(-1,1),)
            self.true_center.append(c)
            #s = random.uniform(0.05,0.5)
            s = 0.05
            x = []
            while len(x) < n:
                # need to change dimension
                lst = []
                for i in c:
                    lst.append(np.random.normal(i,s))
                point = np.array(lst)
                good = True
                for i in point:
                    if abs(i) >= 1:
                        good = False
                if good:
                    x.append(point)
            X.extend(x)
        X = np.array(X)[:self.N]
        return X

    def set_data(self, data):
        self.data_set = data
    # Taking in true center list, and the result center list
    def error_calculate(self,res):
        error = 0.0
        for i in range(self.k):
            min_dis = distance.euclidean(self.true_center[i],res[0])
            for j in range(self.k):
                # calculate the distance btw true center and kmean center
                dst = distance.euclidean(self.true_center[i],res[j])
                # find the minimum dst btw the true center and kmean center
                if dst < min_dis:
                    min_dis = dst
            error += min_dis * min_dis
        # print "the error rate is: ", error
        return math.sqrt(error / (self.sigma * self.sigma * self.p))

    # calculate the true function enables more kmean center than true center
    # this is for the true center 
    def true_cost_function(self):
        cost = 0.0
        # loop through the data set
        for data in self.data_set:
            # min_dist is the distance to the first true center
            min_dis = distance.euclidean(data,self.true_center[0])
            # loop through the true center
            for i in range(self.k):
                 # calculate the distance btw true center and kmean center
                dst = distance.euclidean(data,self.true_center[i])
                # find the minimum dst btw the true center and kmean center
                if dst < min_dis:
                    min_dis = dst
            cost += min_dis * min_dis
        return cost
    
    # this is for the cost function compared to the true center cost function
    def cost_function(self,res):
        cost = 0.0
        # loop through the whole dataset
        for data in self.data_set:
            min_dis = distance.euclidean(data,res[0])
            for i in range(len(res)):
                 # calculate the distance btw true center and kmean center
                dst = distance.euclidean(data,res[i])
                # find the minimum dst btw the true center and kmean center
                if dst < min_dis:
                    min_dis = dst
            cost += min_dis * min_dis
        # the ratio of result center cost/ true center cost
        return cost/self.true_cost_function()
            

    def set_true_center(self, means):
        self.true_center = means
        
        #print "true center is ", self.true_center


    # mean is a p dimensional vector, covarience is a p*p matrix
    # number_of_sample is n samples
    # return value: n*p matrix, generate n samples from N(mean,covarience)
    def generate_mult_normal_data(self, mean, covariance, num_of_samples):
        return np.random.multivariate_normal(mean,covariance, num_of_samples)

    # prob is list of probability that sums to 1
    # means is a p dimensional vector,covarience is a p*p matrix
    # number_of_samples is n samples
    # return value: generate n samples from the mixture of Gaussian
    def generate_mult_normal_based_prob(self, prob, means, covariance, num_of_samples):
        data_list = []
        # keep track which points belong to which cluster
        true_cluster_list = []
        for i in range(num_of_samples):
            # flip a coin to see which cluster the data point comes from
            random_num = random.random()
            sum = 0.0
            index = -1
            for p in prob:
                if random_num < sum:
                    break
                sum += p
                index = index + 1
            true_cluster_list.append(index)
            # call the generate_mult_normal_data rountine
            data_list.append(self.generate_mult_normal_data(means[index], covariance,1)[0])
        self.true_cluster_list = true_cluster_list
        return np.array(data_list)[:num_of_samples]
        #return (np.array(data_list)[:num_of_samples], true_cluster_list)
    def generate_data(self, dimension, p_sigma, number_of_cluster, number_of_points):
        # dimension
        self.p = dimension

        # sqrt of variance
        self.sigma = p_sigma

        # k is number of clusters
        self.k = number_of_cluster

        # n is number of points to be generated
        self.N = number_of_points

        # init a covariance matrix as identity matrix
        self.identity_matrix = np.identity(self.p)

        # covariance matrix
        covariance = self.sigma * self.sigma * self.identity_matrix
        # choose means from


        # means are from a normal distribution with center is all 0s and varience N(0,sigma squre identity matrix)
        mean = []
        for i in range(self.p):
            mean.append(0)
        # choose means, k is the number of centers/cluster
        self.means = self.generate_mult_normal_data(mean, covariance, self.k)

        self.set_true_center(self.means)

        # choose pis, ASA probabolity
        self.PIs = []
        for i in range(self.k):
            # every cluster has the same probability
            self.PIs.append(1.0/self.k)

        # generate n points from the mixture model
        self.data_set = self.generate_mult_normal_based_prob(self.PIs, self.means, covariance, self.N)

In [3]:
from sklearn.datasets import fetch_mldata
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
import matplotlib
import pylab
import matplotlib.pyplot as plt 
mnist = fetch_mldata('MNIST original', data_home='./data')
X_digits, _,_, Y_digits = mnist.values() # fetch dataset from internet
images0 = []
images1 = []
images2 = []
images3 = []
images4 = []
images5 = []
images6 = []
images7 = []
images8 = []
images9 = []
for i in range(len(Y_digits)):
    if Y_digits[i] == 0.0:
        images0.append(X_digits[i])
    elif Y_digits[i] == 1.0:
        images1.append(X_digits[i])
    elif Y_digits[i] == 2.0:
        images2.append(X_digits[i])
    elif Y_digits[i] == 3.0:
        images3.append(X_digits[i])
    elif Y_digits[i] == 4.0:
        images4.append(X_digits[i])
    elif Y_digits[i] == 5.0:
        images5.append(X_digits[i])
    elif Y_digits[i] == 6.0:
        images6.append(X_digits[i])
    elif Y_digits[i] == 7.0:
        images7.append(X_digits[i])
    elif Y_digits[i] == 8.0:
        images8.append(X_digits[i])
    elif Y_digits[i] == 9.0:
        images9.append(X_digits[i])
mnist_true_center1 = []
mnist_true_center = []
mnist_true_center.append(np.array(images0).mean(axis=0).reshape(28,28))
mnist_true_center.append(np.array(images1).mean(axis=0).reshape(28,28))
mnist_true_center.append(np.array(images2).mean(axis=0).reshape(28,28))
mnist_true_center.append(np.array(images3).mean(axis=0).reshape(28,28))
mnist_true_center.append(np.array(images4).mean(axis=0).reshape(28,28))
mnist_true_center.append(np.array(images5).mean(axis=0).reshape(28,28))
mnist_true_center.append(np.array(images6).mean(axis=0).reshape(28,28))
mnist_true_center.append(np.array(images7).mean(axis=0).reshape(28,28))
mnist_true_center.append(np.array(images8).mean(axis=0).reshape(28,28))
mnist_true_center.append(np.array(images9).mean(axis=0).reshape(28,28))

mnist_true_center1.append(np.array(images0).mean(axis=0))
mnist_true_center1.append(np.array(images1).mean(axis=0))
mnist_true_center1.append(np.array(images2).mean(axis=0))
mnist_true_center1.append(np.array(images3).mean(axis=0))
mnist_true_center1.append(np.array(images4).mean(axis=0))
mnist_true_center1.append(np.array(images5).mean(axis=0))
mnist_true_center1.append(np.array(images6).mean(axis=0))
mnist_true_center1.append(np.array(images7).mean(axis=0))
mnist_true_center1.append(np.array(images8).mean(axis=0))
mnist_true_center1.append(np.array(images9).mean(axis=0))


# imshow(mnist_true_center[0], cmap=cm.gray)
# show()
# imshow(mnist_true_center[1], cmap=cm.gray)
# show()
# imshow(mnist_true_center[2], cmap=cm.gray)
# show()
# imshow(mnist_true_center[3], cmap=cm.gray)
# show()
# imshow(mnist_true_center[4], cmap=cm.gray)
# show()
# imshow(mnist_true_center[5], cmap=cm.gray)
# show()
# imshow(mnist_true_center[6], cmap=cm.gray)
# show()
# imshow(mnist_true_center[7], cmap=cm.gray)
# show()
# imshow(mnist_true_center[8], cmap=cm.gray)
# show()
# imshow(mnist_true_center[9], cmap=cm.gray)
# show()

#X_digits, Y_digits = shuffle(X_digits,Y_digits) # shuffle dataset (which is     ordered!)
X_digits = X_digits[-5000:]
# plt.rc("image", cmap="binary") # use black/white palette for plotting
# for i in xrange(15):
#     plt.subplot(3,5,i+1)
#     plt.imshow(X_digits[10+i].reshape(28,28))
#     plt.xticks(())
#     plt.yticks(())
# plt.show()

# # classify to 10 clusters
# kmeans = KMeans(10)
# mu_digits = kmeans.fit(X_digits).cluster_centers_

# plt.figure(figsize=(16,6))
# for i in xrange(2*(mu_digits.shape[0]/2)): # loop over all means
#     plt.subplot(2,mu_digits.shape[0]/2,i+1)
#     plt.imshow(mu_digits[i].reshape(28,28))
#     plt.xticks(())
#     plt.yticks(())
# plt.show()



In [ ]:
print "number of data is ", len(X_digits)
print "each points is in dimension ", len(X_digits[0])
d2 = Data(len(X_digits), 10, len(X_digits[0]))
d2.set_data(X_digits)
d2.set_true_center(mnist_true_center1)
# run 5 times
cost_function_score = []
for i in range (0,100):
    # classify to 10 clusters
    kmeans = KMeans(10)
    print i
    mu_digits = kmeans.fit(X_digits).cluster_centers_
    cost_function_score.append(d2.cost_function(mu_digits))
print cost_function_score
# plot the histogram for the score
plt.xlabel('Cost function results')
plt.ylabel('Frequency of the cost function result')
plt.title(r'Histogram of Kmean on MNIST dataset for 10 clusters')
n, bins, patches = plt.hist(cost_function_score,20,normed=0,facecolor="Red",alpha=0.75)
plt.show()

#print d2.cost_function(mu_digits)

In [ ]:
#cluster MNIST to 15 clusters
# run 5 times
cost_function_score = []
for i in range (0,5):
    # classify to 10 clusters
    kmeans = KMeans(15)
    mu_digits = kmeans.fit(X_digits).cluster_centers_
    cost_function_score.append(d2.cost_function(mu_digits))
print cost_function_score
# plot the histogram for the score
plt.xlabel('Cost function results')
plt.ylabel('Frequency of the cost function result')
plt.title(r'Histogram of Kmean on MNIST dataset for 15 clusters')
n, bins, patches = plt.hist(cost_function_score,20,normed=0,facecolor="Red",alpha=0.75)
plt.show()

